In [333]:
%pip install google-play-scraper
%pip install sastrawi
%pip install pandas numpy matplotlib seaborn google-play-scraper sastrawi wordcloud nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [334]:
%pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [335]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [336]:
%pip install tensorflow
%pip install keras


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [337]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from google_play_scraper import app, reviews_all, reviews, Sort
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Your script logic here


[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True


**1. Proses Scrapping data melalui Google Play Store**

In [338]:
scrapped_data = reviews_all(
    'com.tokopedia.tkpd',
    sleep_milliseconds=0,
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=10
)

In [339]:
import csv
with open('scrapped_data.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["review", "rating"])
    for review in scrapped_data:
        writer.writerow([review['content'], review['score']])
print("Data berhasil disimpan dalam file CSV.")

Data berhasil disimpan dalam file CSV.


**2. Preprocessing Text**

In [340]:
df_review = pd.read_csv('scrapped_data.csv')
df_review.shape
df_review.head()

,review,rating
0,Sistem pengiriman Tokopedia benar benar sangat...,1
1,"Pengiriman di tokopedia ini lama banget, sesam...",4
2,Udah 2x terakhir pesan disini pengirimannya la...,2
3,Berhubung sepertinya masukan di tiap survey sk...,3
4,Sudah berkali-kali bermasalah sama ekspedisiny...,2


In [341]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117000 entries, 0 to 116999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   review  117000 non-null  object
 1   rating  117000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ MB


In [342]:
#konfigurasi

#pandas
pd.options.mode.chained_assignment = None # Menonaktifkan peringatan chaining

#numpy
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas

In [343]:
#menghapus baris kosong
clean_df = df_review.dropna()
clean_df

,review,rating
0,Sistem pengiriman Tokopedia benar benar sangat...,1
1,"Pengiriman di tokopedia ini lama banget, sesam...",4
2,Udah 2x terakhir pesan disini pengirimannya la...,2
3,Berhubung sepertinya masukan di tiap survey sk...,3
4,Sudah berkali-kali bermasalah sama ekspedisiny...,2
...,...,...
116995,"Sudah ke dua kali kecewa di tokped,eksekusi ny...",2
116996,Terimakasih yah suka banget lebih mudah buat b...,5
116997,Makasih atas bantuannya telah membantu menyele...,5
116998,Kami kan belanja memang 1 alamat kenapa transa...,1


In [344]:
#menghapus baris duplikat
clean_df = clean_df.drop_duplicates()

Jumlah_baris_duplikat = clean_df.duplicated().sum()
print("Jumlah baris duplikat:", Jumlah_baris_duplikat)


Jumlah baris duplikat: 0


In [345]:
#Menggunakan regex

def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'\d+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefolding(text):
    text = text.lower() # mengubah teks menjadi huruf kecil
    return text

def tokenizing(text):
    text = word_tokenize(text) # melakukan tokenisasi teks
    return text

def filter(text):
    stop_words = set(stopwords.words('indonesian')) # menghapus stopwords
    text = [word for word in text if word not in stop_words]
    return text

def stemming(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

def sentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [346]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [347]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['review'].apply(clean_text)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefolding)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizing)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filter)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(sentence)

In [348]:
df_tokped = clean_df.head(10000)
df_tokped

,review,rating,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,Sistem pengiriman Tokopedia benar benar sangat...,1,Sistem pengiriman Tokopedia benar benar sangat...,sistem pengiriman tokopedia benar benar sangat...,sistem pengiriman tokopedia benar benar sangat...,"[sistem, pengiriman, tokopedia, benar, benar, ...","[sistem, pengiriman, tokopedia, menjengkelkan,...",sistem pengiriman tokopedia menjengkelkan karn...
1,"Pengiriman di tokopedia ini lama banget, sesam...",4,Pengiriman di tokopedia ini lama banget sesama...,pengiriman di tokopedia ini lama banget sesama...,pengiriman di tokopedia ini lama banget sesama...,"[pengiriman, di, tokopedia, ini, lama, banget,...","[pengiriman, tokopedia, banget, jawa, timur, d...",pengiriman tokopedia banget jawa timur dikirim...
2,Udah 2x terakhir pesan disini pengirimannya la...,2,Udah x terakhir pesan disini pengirimannya lam...,udah x terakhir pesan disini pengirimannya lam...,udah x terakhir pesan disini pengirimannya lam...,"[udah, x, terakhir, pesan, disini, pengirimann...","[udah, x, pesan, pengirimannya, banget, pesan,...",udah x pesan pengirimannya banget pesan barang...
3,Berhubung sepertinya masukan di tiap survey sk...,3,Berhubung sepertinya masukan di tiap survey sk...,berhubung sepertinya masukan di tiap survey sk...,berhubung sepertinya masukan di tiap survey sk...,"[berhubung, sepertinya, masukan, di, tiap, sur...","[berhubung, masukan, survey, skrg, dipertimban...",berhubung masukan survey skrg dipertimbangkan ...
4,Sudah berkali-kali bermasalah sama ekspedisiny...,2,Sudah berkalikali bermasalah sama ekspedisinya...,sudah berkalikali bermasalah sama ekspedisinya...,sudah berkalikali bermasalah sama ekspedisinya...,"[sudah, berkalikali, bermasalah, sama, ekspedi...","[berkalikali, bermasalah, ekspedisinya, pake, ...",berkalikali bermasalah ekspedisinya pake siste...
...,...,...,...,...,...,...,...,...
9999,secara pribadi jujur aja belanja online selama...,5,secara pribadi jujur aja belanja online selama...,secara pribadi jujur aja belanja online selama...,secara pribadi jujur aja belanja online selama...,"[secara, pribadi, jujur, aja, belanja, online,...","[pribadi, jujur, aja, belanja, online, asyik, ...",pribadi jujur aja belanja online asyik nyaman ...
10000,TOKOPEDIA memberikan ku pengalaman belanja ter...,4,TOKOPEDIA memberikan ku pengalaman belanja ter...,tokopedia memberikan ku pengalaman belanja ter...,tokopedia memberikan ku pengalaman belanja ter...,"[tokopedia, memberikan, ku, pengalaman, belanj...","[tokopedia, ku, pengalaman, belanja, terbaik, ...",tokopedia ku pengalaman belanja terbaik kebing...
10001,Agak sedikit kecewa di masalah kualitas gambar...,3,Agak sedikit kecewa di masalah kualitas gambar...,agak sedikit kecewa di masalah kualitas gambar...,agak sedikit kecewa di masalah kualitas gambar...,"[agak, sedikit, kecewa, di, masalah, kualitas,...","[kecewa, kualitas, gambar, sih, aplikasi, nya,...",kecewa kualitas gambar sih aplikasi nya gak ka...
10002,Sekarang kebanyakan notif iklan nya. Sampai ad...,1,Sekarang kebanyakan notif iklan nya Sampai ada...,sekarang kebanyakan notif iklan nya sampai ada...,sekarang kebanyakan notif iklan nya sampai ada...,"[sekarang, kebanyakan, notif, iklan, nya, samp...","[kebanyakan, notif, iklan, nya, notif, barang,...",kebanyakan notif iklan nya notif barang tujuan...


**3. Tahapan Pelabelan dan Ekstraksi Fitur**

In [349]:
from textblob import TextBlob # type: ignore

def analisis_sentimen(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

df_tokped['polarity_textblob'] = df_tokped['text_akhir'].apply(analisis_sentimen)
df_tokped

,review,rating,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_textblob
0,Sistem pengiriman Tokopedia benar benar sangat...,1,Sistem pengiriman Tokopedia benar benar sangat...,sistem pengiriman tokopedia benar benar sangat...,sistem pengiriman tokopedia benar benar sangat...,"[sistem, pengiriman, tokopedia, benar, benar, ...","[sistem, pengiriman, tokopedia, menjengkelkan,...",sistem pengiriman tokopedia menjengkelkan karn...,neutral
1,"Pengiriman di tokopedia ini lama banget, sesam...",4,Pengiriman di tokopedia ini lama banget sesama...,pengiriman di tokopedia ini lama banget sesama...,pengiriman di tokopedia ini lama banget sesama...,"[pengiriman, di, tokopedia, ini, lama, banget,...","[pengiriman, tokopedia, banget, jawa, timur, d...",pengiriman tokopedia banget jawa timur dikirim...,neutral
2,Udah 2x terakhir pesan disini pengirimannya la...,2,Udah x terakhir pesan disini pengirimannya lam...,udah x terakhir pesan disini pengirimannya lam...,udah x terakhir pesan disini pengirimannya lam...,"[udah, x, terakhir, pesan, disini, pengirimann...","[udah, x, pesan, pengirimannya, banget, pesan,...",udah x pesan pengirimannya banget pesan barang...,neutral
3,Berhubung sepertinya masukan di tiap survey sk...,3,Berhubung sepertinya masukan di tiap survey sk...,berhubung sepertinya masukan di tiap survey sk...,berhubung sepertinya masukan di tiap survey sk...,"[berhubung, sepertinya, masukan, di, tiap, sur...","[berhubung, masukan, survey, skrg, dipertimban...",berhubung masukan survey skrg dipertimbangkan ...,neutral
4,Sudah berkali-kali bermasalah sama ekspedisiny...,2,Sudah berkalikali bermasalah sama ekspedisinya...,sudah berkalikali bermasalah sama ekspedisinya...,sudah berkalikali bermasalah sama ekspedisinya...,"[sudah, berkalikali, bermasalah, sama, ekspedi...","[berkalikali, bermasalah, ekspedisinya, pake, ...",berkalikali bermasalah ekspedisinya pake siste...,neutral
...,...,...,...,...,...,...,...,...,...
9999,secara pribadi jujur aja belanja online selama...,5,secara pribadi jujur aja belanja online selama...,secara pribadi jujur aja belanja online selama...,secara pribadi jujur aja belanja online selama...,"[secara, pribadi, jujur, aja, belanja, online,...","[pribadi, jujur, aja, belanja, online, asyik, ...",pribadi jujur aja belanja online asyik nyaman ...,positive
10000,TOKOPEDIA memberikan ku pengalaman belanja ter...,4,TOKOPEDIA memberikan ku pengalaman belanja ter...,tokopedia memberikan ku pengalaman belanja ter...,tokopedia memberikan ku pengalaman belanja ter...,"[tokopedia, memberikan, ku, pengalaman, belanj...","[tokopedia, ku, pengalaman, belanja, terbaik, ...",tokopedia ku pengalaman belanja terbaik kebing...,neutral
10001,Agak sedikit kecewa di masalah kualitas gambar...,3,Agak sedikit kecewa di masalah kualitas gambar...,agak sedikit kecewa di masalah kualitas gambar...,agak sedikit kecewa di masalah kualitas gambar...,"[agak, sedikit, kecewa, di, masalah, kualitas,...","[kecewa, kualitas, gambar, sih, aplikasi, nya,...",kecewa kualitas gambar sih aplikasi nya gak ka...,neutral
10002,Sekarang kebanyakan notif iklan nya. Sampai ad...,1,Sekarang kebanyakan notif iklan nya Sampai ada...,sekarang kebanyakan notif iklan nya sampai ada...,sekarang kebanyakan notif iklan nya sampai ada...,"[sekarang, kebanyakan, notif, iklan, nya, samp...","[kebanyakan, notif, iklan, nya, notif, barang,...",kebanyakan notif iklan nya notif barang tujuan...,neutral


In [350]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [351]:
# Menggunakan VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def analisis_sentimen_vader(text):
    analyzer = SentimentIntensityAnalyzer()
    analysis = analyzer.polarity_scores(text)
    if analysis['compound'] > 0:
        return 'positive'
    elif analysis['compound'] == 0:
        return 'neutral'
    else:
        return 'negative'

df_tokped['polarity_vader'] = df_tokped['text_akhir'].apply(analisis_sentimen_vader)
df_tokped

,review,rating,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_textblob,polarity_vader
0,Sistem pengiriman Tokopedia benar benar sangat...,1,Sistem pengiriman Tokopedia benar benar sangat...,sistem pengiriman tokopedia benar benar sangat...,sistem pengiriman tokopedia benar benar sangat...,"[sistem, pengiriman, tokopedia, benar, benar, ...","[sistem, pengiriman, tokopedia, menjengkelkan,...",sistem pengiriman tokopedia menjengkelkan karn...,neutral,neutral
1,"Pengiriman di tokopedia ini lama banget, sesam...",4,Pengiriman di tokopedia ini lama banget sesama...,pengiriman di tokopedia ini lama banget sesama...,pengiriman di tokopedia ini lama banget sesama...,"[pengiriman, di, tokopedia, ini, lama, banget,...","[pengiriman, tokopedia, banget, jawa, timur, d...",pengiriman tokopedia banget jawa timur dikirim...,neutral,neutral
2,Udah 2x terakhir pesan disini pengirimannya la...,2,Udah x terakhir pesan disini pengirimannya lam...,udah x terakhir pesan disini pengirimannya lam...,udah x terakhir pesan disini pengirimannya lam...,"[udah, x, terakhir, pesan, disini, pengirimann...","[udah, x, pesan, pengirimannya, banget, pesan,...",udah x pesan pengirimannya banget pesan barang...,neutral,neutral
3,Berhubung sepertinya masukan di tiap survey sk...,3,Berhubung sepertinya masukan di tiap survey sk...,berhubung sepertinya masukan di tiap survey sk...,berhubung sepertinya masukan di tiap survey sk...,"[berhubung, sepertinya, masukan, di, tiap, sur...","[berhubung, masukan, survey, skrg, dipertimban...",berhubung masukan survey skrg dipertimbangkan ...,neutral,neutral
4,Sudah berkali-kali bermasalah sama ekspedisiny...,2,Sudah berkalikali bermasalah sama ekspedisinya...,sudah berkalikali bermasalah sama ekspedisinya...,sudah berkalikali bermasalah sama ekspedisinya...,"[sudah, berkalikali, bermasalah, sama, ekspedi...","[berkalikali, bermasalah, ekspedisinya, pake, ...",berkalikali bermasalah ekspedisinya pake siste...,neutral,neutral
...,...,...,...,...,...,...,...,...,...,...
9999,secara pribadi jujur aja belanja online selama...,5,secara pribadi jujur aja belanja online selama...,secara pribadi jujur aja belanja online selama...,secara pribadi jujur aja belanja online selama...,"[secara, pribadi, jujur, aja, belanja, online,...","[pribadi, jujur, aja, belanja, online, asyik, ...",pribadi jujur aja belanja online asyik nyaman ...,positive,positive
10000,TOKOPEDIA memberikan ku pengalaman belanja ter...,4,TOKOPEDIA memberikan ku pengalaman belanja ter...,tokopedia memberikan ku pengalaman belanja ter...,tokopedia memberikan ku pengalaman belanja ter...,"[tokopedia, memberikan, ku, pengalaman, belanj...","[tokopedia, ku, pengalaman, belanja, terbaik, ...",tokopedia ku pengalaman belanja terbaik kebing...,neutral,positive
10001,Agak sedikit kecewa di masalah kualitas gambar...,3,Agak sedikit kecewa di masalah kualitas gambar...,agak sedikit kecewa di masalah kualitas gambar...,agak sedikit kecewa di masalah kualitas gambar...,"[agak, sedikit, kecewa, di, masalah, kualitas,...","[kecewa, kualitas, gambar, sih, aplikasi, nya,...",kecewa kualitas gambar sih aplikasi nya gak ka...,neutral,neutral
10002,Sekarang kebanyakan notif iklan nya. Sampai ad...,1,Sekarang kebanyakan notif iklan nya Sampai ada...,sekarang kebanyakan notif iklan nya sampai ada...,sekarang kebanyakan notif iklan nya sampai ada...,"[sekarang, kebanyakan, notif, iklan, nya, samp...","[kebanyakan, notif, iklan, nya, notif, barang,...",kebanyakan notif iklan nya notif barang tujuan...,neutral,neutral


In [352]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [353]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    # buat polarity neutral, positive, negative
    if score > 1:
        polarity = 'positive'
    elif score < -1:
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [354]:
results = df_tokped['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df_tokped['polarity_score'] = results[0]
df_tokped['polarity'] = results[1]
print(df_tokped['polarity'].value_counts())

polarity
negative    4430
positive    4096
neutral     1474
Name: count, dtype: int64


In [355]:
df_tokped.tail(300)

,review,rating,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_textblob,polarity_vader,polarity_score,polarity
9703,Mengecewakan... Pengen cepat malah berantakan....,1,Mengecewakan Pengen cepat malah berantakan Pes...,mengecewakan pengen cepat malah berantakan pes...,mengecewakan pengen cepat malah berantakan pes...,"[mengecewakan, pengen, cepat, malah, berantaka...","[mengecewakan, pengen, cepat, berantakan, pese...",mengecewakan pengen cepat berantakan pesen pak...,neutral,positive,-3,negative
9704,Mending shopee ya cashbacknya JELAS. Kalau di ...,1,Mending shopee ya cashbacknya JELAS Kalau di t...,mending shopee ya cashbacknya jelas kalau di t...,mending shopee ya cashbacknya jelas kalau di t...,"[mending, shopee, ya, cashbacknya, jelas, kala...","[mending, shopee, ya, cashbacknya, tokopedia, ...",mending shopee ya cashbacknya tokopedia beli p...,neutral,neutral,7,positive
9705,Mohon maaf sebelumnya kali ini saya sangat men...,1,Mohon maaf sebelumnya kali ini saya sangat men...,mohon maaf sebelumnya kali ini saya sangat men...,mohon maaf sebelumnya kali ini saya sangat men...,"[mohon, maaf, sebelumnya, kali, ini, saya, san...","[mohon, maaf, kali, menyayangkan, kurir, rekom...",mohon maaf kali menyayangkan kurir rekomendasi...,neutral,neutral,0,neutral
9706,Alhmduliilah sangat meyenangakan dan sangat me...,5,Alhmduliilah sangat meyenangakan dan sangat me...,alhmduliilah sangat meyenangakan dan sangat me...,alhmduliilah sangat meyenangakan dan sangat me...,"[alhmduliilah, sangat, meyenangakan, dan, sang...","[alhmduliilah, meyenangakan, membantu, alhamdu...",alhmduliilah meyenangakan membantu alhamdulill...,neutral,neutral,18,positive
9707,Saya senang berbelanja di Tokopedia Semoga Tok...,5,Saya senang berbelanja di Tokopedia Semoga Tok...,saya senang berbelanja di tokopedia semoga tok...,saya senang berbelanja di tokopedia semoga tok...,"[saya, senang, berbelanja, di, tokopedia, semo...","[senang, berbelanja, tokopedia, semoga, tokope...",senang berbelanja tokopedia semoga tokopedia b...,neutral,neutral,-17,negative
...,...,...,...,...,...,...,...,...,...,...,...,...
9999,secara pribadi jujur aja belanja online selama...,5,secara pribadi jujur aja belanja online selama...,secara pribadi jujur aja belanja online selama...,secara pribadi jujur aja belanja online selama...,"[secara, pribadi, jujur, aja, belanja, online,...","[pribadi, jujur, aja, belanja, online, asyik, ...",pribadi jujur aja belanja online asyik nyaman ...,positive,positive,25,positive
10000,TOKOPEDIA memberikan ku pengalaman belanja ter...,4,TOKOPEDIA memberikan ku pengalaman belanja ter...,tokopedia memberikan ku pengalaman belanja ter...,tokopedia memberikan ku pengalaman belanja ter...,"[tokopedia, memberikan, ku, pengalaman, belanj...","[tokopedia, ku, pengalaman, belanja, terbaik, ...",tokopedia ku pengalaman belanja terbaik kebing...,neutral,positive,4,positive
10001,Agak sedikit kecewa di masalah kualitas gambar...,3,Agak sedikit kecewa di masalah kualitas gambar...,agak sedikit kecewa di masalah kualitas gambar...,agak sedikit kecewa di masalah kualitas gambar...,"[agak, sedikit, kecewa, di, masalah, kualitas,...","[kecewa, kualitas, gambar, sih, aplikasi, nya,...",kecewa kualitas gambar sih aplikasi nya gak ka...,neutral,neutral,5,positive
10002,Sekarang kebanyakan notif iklan nya. Sampai ad...,1,Sekarang kebanyakan notif iklan nya Sampai ada...,sekarang kebanyakan notif iklan nya sampai ada...,sekarang kebanyakan notif iklan nya sampai ada...,"[sekarang, kebanyakan, notif, iklan, nya, samp...","[kebanyakan, notif, iklan, nya, notif, barang,...",kebanyakan notif iklan nya notif barang tujuan...,neutral,neutral,-8,negative


In [420]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df_tokped['text_akhir']
# y = df_tokped['polarity']
y = df_tokped['polarity_textblob']
# y = df_tokped['polarity_vader']


In [421]:
# Encode label
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=100, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [422]:
features_df

,aja,akun,apk,aplikasi,bagus,banget,barang,batalkan,bayar,belanja,...,toped,tp,transaksi,uang,udah,udh,update,voucher,ya,yg
0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.157740,0.0,0.000000,0.170872,0.126015,0.0,0.0,0.0,0.000000,0.286366
1,0.173682,0.0,0.0,0.000000,0.0,0.175417,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.137982
2,0.000000,0.0,0.0,0.180092,0.0,0.424599,0.176209,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.440914,0.0,0.0,0.0,0.000000,0.000000
3,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.491912
4,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.214533,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.181190,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.162561,...,0.000000,0.0,0.000000,0.000000,0.190032,0.0,0.0,0.0,0.000000,0.000000
9996,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.231988,0.514240,...,0.000000,0.0,0.215906,0.000000,0.000000,0.0,0.0,0.0,0.671538,0.000000
9997,0.000000,0.0,0.0,0.277796,0.0,0.000000,0.271808,0.0,0.000000,0.290901,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
9998,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.274770,0.0,0.000000,0.000000,...,0.215156,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [423]:
# Ektrasi fitur dengan metode Word2Vec
from gensim.models import Word2Vec

# Tokenisasi teks
tokenized_text = [word_tokenize(text) for text in df_tokped['text_akhir']]
# Membuat model Word2Vec
word2vec = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, sg=1)

# Membuat vektor fitur Word2Vec
X_word2vec = np.array([np.mean([word2vec.wv[word] for word in document if word in word2vec.wv] or [np.zeros(100)], axis=0) for document in tokenized_text])

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_word2vec)

In [424]:
# Ekstraksi fitur dengan metode Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Membuat dokumen berlabel
tagged_data = [TaggedDocument(words=word_tokenize(text), tags=[str(i)]) for i, text in enumerate(df_tokped['text_akhir'])]

# Membuat model Doc2Vec
doc2vec = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=100)
doc2vec.build_vocab(tagged_data)
doc2vec.train(tagged_data, total_examples=doc2vec.corpus_count, epochs=doc2vec.epochs)

# Membuat vektor fitur Doc2Vec
X_doc2vec = np.array([doc2vec.infer_vector(word_tokenize(text)) for text in df_tokped['text_akhir']])

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_doc2vec)

In [425]:
features_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.054969,0.312980,-0.876511,-1.076542,-0.509080,1.164749,-0.482250,-0.527265,0.128616,1.640731,...,-2.538453,0.936472,-1.121200,-1.492423,-0.125536,-1.109596,2.979347,0.870280,-0.177202,-1.264374
1,-0.274030,-1.811528,-1.402435,1.788582,0.503641,-0.179542,-0.374630,-0.647906,-0.168674,-0.250093,...,-0.938197,1.165280,0.261199,-1.156913,1.638115,0.235417,0.261275,-1.294377,-1.168262,-0.799079
2,-0.603133,0.117314,-0.831973,-0.853228,0.200867,-0.673527,1.052961,0.379490,-1.166540,-0.298936,...,-0.936116,-0.219664,0.374660,-0.530028,1.134826,0.638684,1.117223,-0.297613,-0.012926,-1.688185
3,-0.594313,0.155967,0.452207,-0.245501,0.266600,1.374258,0.278876,1.230525,-0.676829,0.013182,...,-0.874472,0.018832,-0.769803,-0.440844,0.538250,0.026256,-1.344690,-0.282895,0.186900,-0.455707
4,-0.495889,-0.641589,0.378454,0.865401,0.063714,0.782770,0.886063,1.317973,-1.805284,-0.204557,...,1.014165,-0.964776,0.004750,-1.251862,0.380249,0.347382,-0.552221,0.760440,0.590741,-0.448010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.364300,0.813153,-0.180682,1.472279,-0.429656,0.104076,-0.517224,1.289818,-0.996081,0.933250,...,1.897088,0.331737,0.818758,0.462590,0.123473,-0.510848,0.493427,-0.407156,0.573602,-0.477371
9996,0.615016,0.724709,-0.977628,-0.626406,-0.832869,-0.623788,-0.339665,0.653384,0.174190,0.953848,...,0.829473,0.870498,-0.533876,-0.181012,0.643153,1.378428,-0.806303,0.042012,0.913618,-1.252511
9997,-1.035465,0.319916,0.019445,0.569091,0.569070,-0.587277,1.891125,-0.230821,-0.126797,-0.157630,...,-0.422637,-0.053999,-0.355577,0.450026,0.062962,0.627379,0.701763,-0.315395,-0.463214,-0.132425
9998,-1.363315,-0.133166,0.963513,0.276222,-0.475698,-1.175315,-0.494670,1.305530,-0.244841,-0.306472,...,0.185616,1.732433,0.354974,0.277437,-0.883735,-0.251320,0.989667,-0.031162,0.545985,0.861007


**4. Pelatihan dan Evaluasi Model**

In [426]:
X_train, X_test, y_train, y_test = train_test_split(features_df, y, test_size=0.2, random_state=42)


In [427]:
# distribusi label
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{'negative': 247, 'neutral': 7077, 'positive': 676}

In [428]:
from imblearn.over_sampling import SMOTE

# Seimbangkan kelas dengan SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# distribusi label
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{'negative': 7077, 'neutral': 7077, 'positive': 7077}

In [429]:
from sklearn.ensemble import RandomForestClassifier

# Membuat objek model Random Forest
random_forest = RandomForestClassifier(n_estimators=500, random_state=42)

# Melatih model Random Forest pada data pelatihan
# random_forest.fit(X_train.toarray(), y_train)
random_forest.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
# y_pred_train_rf = random_forest.predict(X_train.toarray())
# y_pred_test_rf = random_forest.predict(X_test.toarray())

y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)


# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 1.0
Random Forest - accuracy_test: 0.8615


In [430]:
import joblib
joblib.dump(random_forest, 'model_random_forest.pkl')

['model_random_forest.pkl']

In [431]:
# inference
random_forest = joblib.load('model_random_forest.pkl')

def predict_sentiment(text):
    # Membersihkan teks
    text_clean = clean_text(text)
    text_casefolding = casefolding(text_clean)
    text_slangwords = fix_slangwords(text_casefolding)
    text_tokenizing = tokenizing(text_slangwords)
    text_stopword = filter(text_tokenizing)
    text_akhir = sentence(text_stopword)

    # Ekstraksi fitur
    text_tfidf = tfidf.transform([text_akhir])
   

    # Prediksi sentimen

    sentiment_rf = random_forest.predict(text_tfidf.toarray())

    return sentiment_rf

text = 'Aplikasi ini sangat membantu saya dalam mengelola keuangan. Sangat direkomendasikan!'
# text = 'Aplikasi ini jele÷k!'
sentiment_rf = predict_sentiment(text)

print('Random Forest:', sentiment_rf)



Random Forest: ['neutral']


In [432]:
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

In [433]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=42, max_depth=3)
decision_tree.fit(X_train, y_train)
y_pred_train_dt = decision_tree.predict(X_train)
y_pred_test_dt = decision_tree.predict(X_test)
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

# Menampilkan akurasi
print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)

Decision Tree - accuracy_train: 0.884875
Decision Tree - accuracy_test: 0.8875


In [434]:
import joblib
joblib.dump(decision_tree, 'model_decision_tree.pkl')

['model_decision_tree.pkl']

In [436]:
# inference
decision_tree = joblib.load('model_decision_tree.pkl')

def predict_sentiment(text):
    # Membersihkan teks
    text_clean = clean_text(text)
    text_casefolding = casefolding(text_clean)
    text_slangwords = fix_slangwords(text_casefolding)
    text_tokenizing = tokenizing(text_slangwords)
    text_stopword = filter(text_tokenizing)
    text_akhir = sentence(text_stopword)

    # Ekstraksi fitur
    text_word2vec = np.array([np.mean([word2vec.wv[word] for word in word_tokenize(text_akhir) if word in word2vec.wv] or [np.zeros(100)], axis=0)
                              for document in [text_akhir]])

    # Prediksi sentimen
    sentiment_dt = decision_tree.predict(text_word2vec)

    return sentiment_dt

# text = 'Aplikasi ini sangat membantu saya dalam mengelola keuangan. Sangat direkomendasikan!'
text = 'Aplikasi ini sangat buruk, tidak bisa digunakan. Saya sangat kecewa!'
sentiment_dt = predict_sentiment(text)

# print('Decision Tree:', label_encoder.inverse_transform(sentiment_dt))
print('Decision Tree:', sentiment_dt)

Decision Tree: ['neutral']


In [437]:
# TRAINING MODEL MENGGUNAKAN LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import joblib

# Pisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_doc2vec, y, test_size=0.2, random_state=42)

# Encode label
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

joblib.dump(label_encoder, 'label_encoder.pkl')

# distribusi label
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

# Membuat model RNN
model = Sequential()
model.add(Embedding(input_dim=X_train.shape[0], output_dim=1028, input_length=X_train.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(3, activation='softmax'))

# Kompilasi model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Membuat objek EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Melatih model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluasi model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

Epoch 1/10


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


200/200 ━━━━━━━━━━━━━━━━━━━━ 53s 250ms/step - accuracy: 0.8838 - loss: 0.4633 - val_accuracy: 0.8825 - val_loss: 0.4253
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 49s 244ms/step - accuracy: 0.8856 - loss: 0.4192 - val_accuracy: 0.8825 - val_loss: 0.4252
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 48s 242ms/step - accuracy: 0.8905 - loss: 0.4074 - val_accuracy: 0.8825 - val_loss: 0.4279
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 48s 243ms/step - accuracy: 0.8834 - loss: 0.4227 - val_accuracy: 0.8825 - val_loss: 0.4270
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.8819 - loss: 0.4274
Accuracy: 0.887499988079071


In [438]:
#Simpan model hasil training

#lstm model
model.save('sentiment_lstm_model.h5')

In [440]:
# inference model lstm
from tensorflow.keras.models import load_model

model = load_model('sentiment_lstm_model.h5')
label_encoder = joblib.load('label_encoder.pkl')


def predict_sentiment_lstm(text):
    # Membersihkan teks
    text_clean = clean_text(text)
    text_casefolding = casefolding(text_clean)
    text_slangwords = fix_slangwords(text_casefolding)
    text_tokenizing = tokenizing(text_slangwords)
    text_stopword = filter(text_tokenizing)
    text_akhir = sentence(text_stopword)

    # Ekstraksi fitur
    text_doc2vec = np.array([np.mean([doc2vec.wv[word] for word in word_tokenize(text_akhir) if word in doc2vec.wv] or [np.zeros(100)], axis=0)
                              for document in [text_akhir]])

    # Prediksi sentimen
    sentiment_probs = model.predict(text_doc2vec)
    sentiment_lstm = np.argmax(sentiment_probs, axis=1)

    return sentiment_lstm, sentiment_probs

text = 'Aplikasi ini sangat buruk, tidak bisa digunakan. Saya sangat kecewa!'
sentiment_lstm, sentiment_probs = predict_sentiment_lstm(text)
print(sentiment_probs)
print('LSTM:', label_encoder.inverse_transform(sentiment_lstm))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
[[0.0358106 0.8735267 0.0906626]]
LSTM: ['neutral']
